In [1]:
import numpy as np
import pandas as pd
import time
import datetime
import random
import gc
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from scipy import sparse
import lightgbm as lgb
import xgboost as xgb
from sklearn.ensemble import GradientBoostingClassifier as GBC
from sklearn.metrics import log_loss
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

C:\Users\fc751\AppData\Roaming\Python\Python37\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
#path = './data/'
#train = pd.read_table(path+"round1_ijcai_18_train_20180301.txt",sep=" ")
#test = pd.read_table(path+'round1_ijcai_18_test_a_20180301.txt',sep=" ")
#data = train.append(test, ignore_index=True)
#data.sort_values(by='context_timestamp') #时间戳排序
#train = pd.read_csv(path+"2train.csv")
#test = pd.read_csv(path+"2test.csv")

In [3]:
data = pd.read_csv("./data/train.csv")
data['min_lab'] = data['hour'] * 60 + data['minute']
data.sort_values(by=['user_id','min_lab'])
min_query = data.groupby(['user_id', 'min_lab'], as_index=False)['instance_id'].agg({'min_query': 'count'})

FileNotFoundError: File b'./data/train.csv' does not exist

In [ ]:
data.groupby(['user_gender_id', 'item_brand_id', 'day'], as_index=False)['hour'].agg({'user_day_hour_item_var': 'var'})

In [ ]:
temp = data[['instance_id','item_id']].loc[data.hour>10]

In [ ]:
sub = pd.read_csv("./submit/lgb_feat_imp.csv")
low = sub.loc[sub.score<20]
low.sort_values('feature')

In [ ]:
low = sub.loc[(sub.score>=0)&(sub.score<=200)]
#low.sort_values('feature')
low.loc[low.feature=='click_user_city_lab']

In [ ]:
shop_item

In [ ]:
df.head()

In [ ]:
def rolling(df,roll,window):
    last=roll(df,window=window).tolist()
    last=[np.NaN]+last
    last.pop()
    return last
uid = min_query['user_id'].unique().tolist()
new = pd.DataFrame()
for u in uid:
    print(u)
    df = min_query[min_query['user_id'] == u]
    df['last_1_sum_user'] = rolling(df['min_query'], pd.rolling_sum, window=1)
    df['last_3_sum_user'] = rolling(df['min_query'], pd.rolling_sum, window=3)
    df['last_5_sum_user'] = rolling(df['min_query'], pd.rolling_sum, window=5)
    df['last_10_sum_user'] = rolling(df['min_query'], pd.rolling_sum, window=10)
    df['last_15_sum_user'] = rolling(df['min_query'], pd.rolling_sum, window=15)
    
    new = pd.concat([new, df])

In [ ]:
min15_click_all = pd.DataFrame()
for index, row in data.iterrows():
    print(index)
    start_date = row['min_lab'] - 15
    user_id = row['user_id']
    shop_id = row['shop_id']
    gap_data = data[(data.min_lab > start_date) & (data.min_lab <= row['min_lab'])]
    gap_data = gap_data[(gap_data.user_id == user_id) & (gap_data.shop_id == shop_id)]
    min15_click = gap_data.groupby(['user_id', 'shop_id']).size().reset_index().rename(columns={0:'min15_user_shop_click'})
    min15_click_all = pd.concat([min15_click_all, min15_click], axis=0)
    min15_click_all = min15_click_all.reset_index(drop=True)
min15_click_all = min15_click_all['min15_user_shop_click']
data=pd.concat([data, min15_user_shop_click, 'left'], axis=1)
print('min 15 len :%d' % (len(data)))

In [ ]:
data['min10'] = data['min'] // 10
data['min15'] = data['min'] // 15
data['min30'] = data['min'] // 30
data['min45'] = data['min'] // 45

min10_user_click = data.groupby(['user_id', 'min_10']).size().reset_index().rename(columns={0:'min10_user_click'})
min15_user_click = data.groupby(['user_id', 'min_15']).size().reset_index().rename(columns={0:'min15_user_click'})
min30_user_click = data.groupby(['user_id', 'min_30']).size().reset_index().rename(columns={0:'min30_user_click'})
min45_user_click = data.groupby(['user_id', 'min_45']).size().reset_index().rename(columns={0:'min45_user_click'})

data = pd.merge(data, min10_user_click, 'left', on=['user_id', 'min_10'])
data = pd.merge(data, min15_user_click, 'left', on=['user_id', 'min_15'])
data = pd.merge(data, min30_user_click, 'left', on=['user_id', 'min_30'])
data = pd.merge(data, min45_user_click, 'left', on=['user_id', 'min_45'])

In [ ]:
test = pd.read_csv("./data/final_test_04-30-22-11.csv")
test.shape

In [ ]:
cols = test.columns.tolist()
for c in cols:
    print(c)
    print(test[[c]].info())

In [ ]:
data['time'] = pd.to_datetime(data.context_timestamp, unit='s')
data['time'] = data['time'].apply(lambda x: x + datetime.timedelta(hours=8))
data['day'] = data['time'].apply(lambda x: int(str(x)[8:10]))
data['hour'] = data['time'].apply(lambda x: int(str(x)[11:13])) 

In [ ]:
coll_query = data.groupby(['day', 'hour', 'minute'], as_index=False)['item_collected_level'].agg({'hour_query_collect': 'sum'})
pv_query = data.groupby(['day', 'hour', 'minute'], as_index=False)['item_pv_level'].agg({'hour_query_pv': 'sum'})
sales_query = data.groupby(['day', 'hour', 'minute'], as_index=False)['item_sales_level'].agg({'hour_query_sales': 'sum'})
trade_cnt = data.groupby(['day', 'hour', 'minute'], as_index=False)['is_trade'].agg({'click': 'count'})
istrade_cnt = data.groupby(['day', 'hour', 'minute'], as_index=False)['is_trade'].agg({'trade': 'sum'})

coll_query = coll_query.merge(pv_query, how='left', on=['day', 'hour', 'minute'])
coll_query = coll_query.merge(sales_query, how='left', on=['day', 'hour', 'minute'])
coll_query = coll_query.merge(trade_cnt, how='left', on=['day', 'hour', 'minute'])
coll_query = coll_query.merge(istrade_cnt, how='left', on=['day', 'hour', 'minute'])

coll_query['coll_sales'] = round(coll_query['hour_query_collect'] / coll_query['hour_query_sales'],4)
coll_query['pv_sales'] = round(coll_query['hour_query_pv'] / coll_query['hour_query_sales'], 4)
coll_query['ctr'] = round(coll_query['trade'] / coll_query['click'], 4)

In [ ]:
data['shop_score_service'].unique()

In [ ]:
data.loc[data.user_id==419885]

In [ ]:
temp = coll_query.loc[coll_query.hour==0]
line = plt.plot(range(len(temp)),temp['ctr'],label=u'ctr')
plt.legend()
plt.show()

In [ ]:
temp = coll_query.loc[coll_query.hour==1]
line = plt.plot(range(len(temp)),temp['ctr'],label=u'ctr')
plt.legend()
plt.show()

In [ ]:
temp = coll_query.loc[coll_query.hour==2]
line = plt.plot(range(len(temp)),temp['hour_query_pv'],label=u'hour_query_pv')
plt.legend()
plt.show()

In [ ]:
temp = pv_query.loc[pv_query.hour==11]
line = plt.plot(range(len(temp)),temp['hour_query_pv'],label=u'hour_query_pv')
plt.legend()
plt.show()

In [ ]:
temp = pv_query.loc[pv_query.hour==0]
line = plt.plot(range(len(temp)),temp['hour_query_pv'],label=u'hour_query_pv')
plt.legend()
plt.show()

In [ ]:
line = plt.plot(range(len(data)), (data['hour_query_price']),label=u'price')
plt.legend()
plt.show()

In [ ]:
def feat_select(train, test):

    #去除全局特征instance_id, context_id, user_id
    features = train.drop(['is_trade', 'instance_id', 'time', 'context_timestamp'], axis=1).columns.tolist()   
    target = ['is_trade']

    return features, target

In [ ]:
def lgb_model():

    print('LGBMClassifier...')
    clf = lgb.LGBMClassifier(boosting_type='gbdt', num_leaves=29, max_depth=-1, learning_rate=0.1, n_estimators=100,
                               max_bin=425, subsample_for_bin=50000, objective='binary', min_split_gain=0,
                               min_child_weight=5, min_child_samples=10, subsample=1, subsample_freq=1,
                               colsample_bytree=1, reg_alpha=3, reg_lambda=5, seed=1000, nthread=-1, silent=True)
    return clf

In [ ]:
data.shape

In [ ]:
train = data[:train.shape[0]]
test = data[train.shape[0]:]
del data
gc.collect()
print(train.shape)
print(test.shape)

In [ ]:
def find_best_feat(now_feature, train, test):
    features, target = feat_select(train, test)

    train_x = train.loc[(train.day < 24) & (train.day > 18)]# 19,20,21,22,23
    test_x = train.loc[train.day == 24]
    
    clf = lgb_model()
    
    clf.fit(train_x[now_feature], train_x[target])
    test_x['lgb_predict'] = clf.predict_proba(test_x[now_feature])[:, 1]
    res = log_loss(test_x[target], test_x['lgb_predict'])

    return res


In [ ]:
#正向搜索
features, target = feat_select(train, test)
now_feature = []
check = 100
le = len(features)
for i in range(le):
    now_feature.append(features[i])
    jj = find_best_feat(now_feature, train, test)
    if (jj<check)&(check-jj>0.00003):
        print('目前特征长度为',len(now_feature),' 目前帅气的logloss值是',jj,' 成功加入第',i+1,'个','降低为',check-jj)
        check = jj
    else:
        now_feature.pop()

In [ ]:
#反向搜索
features, target = feat_select(train, test)
now_feature_1 = []
check = 100
le = len(features)
for i in range(le):
    now_feature_1.append(features[le-1-i])
    jj = find_best_feat(now_feature_1, train, test)
    if (jj<check)&(check-jj>0.00003):
        print('目前特征长度为',len(now_feature_1),' 目前帅气的logloss值是',jj,' 成功加入第',i+1,'个','降低为',check-jj)
        check = jj
    else:
        now_feature_1.pop()

In [ ]:
#随机选取特征进行搜索
import random
feat, target = feat_select(train, test)
le = len(feat)
m = le 
del_feat = []
s = 0
while(s<10):
    print('------------------------第',s+1,'轮-------------------------')
    features, target = feat_select(train, test)
    temp = []
    check = 0.1
    m = le
    for i in range(le):
        n = random.randint(0, m-1)
        temp.append(features[n])
        jj = find_best_feat(temp, train, test)
        if (jj<check)&(check-jj>0.00003):
            print('目前特征长度为',len(temp),' 目前帅气的logloss值是',jj,' 成功加入第',i+1,'个','降低为',check-jj)
            check = jj
            del_feat.append(features[n])
        else:
            temp.pop()
        features.remove(features[n])
        m -= 1
    s += 1
    for f in del_feat:
        if f in feat:
            feat.remove(f)
    print('---------------------剩余特征',len(feat),'个-----------------')

In [ ]:
#从中间开始搜索
features, target = feat_select(train, test)
now_feature_3 = []
check = 100
le = len(features)
lo = int(le/2)
hi = lo + 1
j = 0
for i in range(int(le/2)):
    if(lo >= 0):
        now_feature_3.append(features[lo])
        jj = find_best_feat(now_feature_3, train, test)
        if (jj<check)&(check-jj>0.00003):
            print('目前特征长度为',len(now_feature_3),' 目前帅气的logloss值是',jj,' 成功加入第',j+1,'个','降低为',check-jj)
            check = jj
        else:
            now_feature_3.pop()
        lo -= 1
        j += 1
    if(hi < le):
        now_feature_3.append(features[hi])
        jj = find_best_feat(now_feature_3, train, test)
        if (jj<check)&(check-jj>0.00003):
            print('目前特征长度为',len(now_feature_3),' 目前帅气的logloss值是',jj,' 成功加入第',j+1,'个','降低为',check-jj)
            check = jj
        else:
            now_feature_3.pop()
        hi += 1
        j += 1

In [ ]:
li = now_feature + now_feature_1 + now_feature_2 + now_feature_3

In [ ]:
#小号保存
li = set(li)
li = list(li)
len(li)

In [ ]:
print(find_best_feat(features, train, test))

In [ ]:
features, target = feat_select(train, test)
for f in feat:
    if(f in features):
        features.remove(f)

In [ ]:
features

In [ ]:
li.sort()

In [ ]:
features

In [ ]:
import numpy
a = [[0.000125, 2, 3, 4, 5],
     [1, 2, 3, 4, 5],
     [1, 2, 3, 4, 5]
    ]
numpy.savetxt('arr.txt',a)
b = numpy.loadtxt('arr.txt')

In [ ]:
b = numpy.loadtxt('arr.txt')

In [ ]:
b.tolist()